In [ ]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from data import get_prices, get_trades
from plotly.subplots import make_subplots
from randomcolor import RandomColor
from typing import Any

In [ ]:
def get_popular_price(row: Any, bid_ask: str) -> int:
    best_price = -1
    max_volume = -1

    for i in range(1, 4):
        volume = getattr(row, f"{bid_ask}_volume_{i}")
        if math.isnan(volume):
            break

        if volume > max_volume:
            best_price = getattr(row, f"{bid_ask}_price_{i}")
            max_volume = volume

    return best_price

def get_product_prices(prices: pd.DataFrame, product: str) -> np.ndarray:
    prices = prices[prices["product"] == product]

    mid_prices = []
    for row in prices.itertuples():
        popular_buy_price = get_popular_price(row, "bid")
        popular_sell_price = get_popular_price(row, "ask")
        mid_price = (popular_buy_price + popular_sell_price) / 2
        mid_prices.append(mid_price)

    return np.array(mid_prices)

In [ ]:
days = [[5, [2, 3, 4]]]

# manually eyeball trader names just for sanity's sake
traders = [
    "Caesar", "Camilla", "Charlie",
    "Gary", "Gina",
    "Olga", "Olivia",
    "Pablo", "Paris", "Penelope", "Peter"
]

# emptyset to fill
traders = set()

In [ ]:
# get unique list of traders
for round_num, day_nums in days:
    for day_num in day_nums:
        trades = get_trades(round_num, day_num)
        traders = traders.union(trades["buyer"])
        traders = traders.union(trades["seller"])

traders = sorted(traders)
traders

In [ ]:
combinations = set()

# get all combinations of buyer-seller
for round_num, day_nums in days:
    for day_num in day_nums:
        trades = get_trades(round_num, day_num)
        for row in trades.itertuples():
            combinations.add((row.buyer, row.seller))

sorted(combinations)

In [ ]:
# plot product prices and markers of buyers and sellers
for round_num, day_nums in days:
    prices = pd.DataFrame()
    trades = pd.DataFrame()

    for day_num in day_nums:
        if len(prices) == 0:
            timestamp_offset = 0
        else:
            timestamp_offset = int(prices["timestamp"].tail(1).iloc[0])

        day_prices = get_prices(round_num, day_num)
        day_trades = get_trades(round_num, day_num)

        day_prices["timestamp"] += timestamp_offset
        day_trades["timestamp"] += timestamp_offset

        prices = pd.concat([prices, day_prices])
        trades = pd.concat([trades, day_trades])

    for product in sorted(prices["product"].unique()):
        product_prices = get_product_prices(prices, product)
        product_trades = trades[trades["symbol"] == product]

        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(x=prices[prices["product"] == product]["timestamp"], y=product_prices, name="Price", line={"color": "gray"}))

        colors = RandomColor(seed=0)

        for trader in traders:
            for side in ["buyer", "seller"]:
                trader_trades = product_trades[product_trades[side] == trader]
                fig.add_trace(go.Scatter(
                    x=trader_trades["timestamp"],
                    y=trader_trades["price"],
                    mode="markers",
                    name=f"{side} = {trader}",
                    visible="legendonly",
                    line={"color": colors.generate()[0]},
                ))

        fig.update_layout(title_text=f"Round {round_num} - {product}")
        fig.show()

In [ ]:
# plot product prices and markers of buyer-seller pairs
for round_num, day_nums in days:
    prices = pd.DataFrame()
    trades = pd.DataFrame()

    for day_num in day_nums:
        if len(prices) == 0:
            timestamp_offset = 0
        else:
            timestamp_offset = int(prices["timestamp"].tail(1).iloc[0])

        day_prices = get_prices(round_num, day_num)
        day_trades = get_trades(round_num, day_num)

        day_prices["timestamp"] += timestamp_offset
        day_trades["timestamp"] += timestamp_offset

        prices = pd.concat([prices, day_prices])
        trades = pd.concat([trades, day_trades])

    for product in sorted(prices["product"].unique()):
        product_prices = get_product_prices(prices, product)
        product_trades = trades[trades["symbol"] == product]

        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(x=prices[prices["product"] == product]["timestamp"], y=product_prices, name="Price", line={"color": "gray"}))

        colors = RandomColor(seed=0)

        for buyer in traders:
            for seller in traders:
                trader_trades = product_trades[(product_trades["buyer"] == buyer) & (product_trades["seller"] == seller)]
                if len(trader_trades) < 2 or len(trader_trades) > 100:
                    continue

                fig.add_trace(go.Scatter(
                    x=trader_trades["timestamp"],
                    y=trader_trades["price"],
                    mode="markers",
                    name=f"{buyer}/{seller}",
                    visible="legendonly",
                    line={"color": colors.generate()[0]},
                ))

        fig.update_layout(title_text=f"Round {round_num} - {product}")
        fig.show()